In [2]:
!/anaconda/envs/azureml_py310_sdkv2/bin/python -m pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /anaconda/envs/azureml_py310_sdkv2/bin/python -m pip install --upgrade pip


In [3]:
import pandas as pd

# Read your Excel file
df = pd.read_excel('got_persona_dataset_100.xlsx')

# Clean data
df = df.dropna(how='all', axis=0).dropna(how='all', axis=1)
df.columns = df.columns.str.strip().str.replace('[^A-Za-z0-9_]', '_', regex=True)

# Save as CSV
df.to_csv('got_persona_dataset_100.csv', index=False, encoding='utf-8')
print(f"✓ Converted! Shape: {df.shape}")

✓ Converted! Shape: (100, 21)


In [4]:
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()
datastore = ws.get_default_datastore()

# Upload and register
datastore.upload_files(['got_persona_dataset_100.csv'], target_path='datasets/', overwrite=True)
from azureml.data.datapath import DataPath
dataset = Dataset.Tabular.from_delimited_files(DataPath(datastore, 'datasets/got_persona_dataset_100.csv'))
dataset.register(ws, 'got-persona-dataset', create_new_version=True)

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/__init__.py:41: UserWarning: Versions of mlflow (3.1.1) and mlflow-skinny (2.22.1) are different. This may lead to unexpected behavior. Please install the same version of both packages.
  mlflow.mismatch._check_version_mismatch()
Message: [NOT_SUPPORTED_API_USE_ATTEMPT] The [_get_steps] API has been deprecated and is no longer supported
Payload: {"pid": 5511, "rslex_version": "2.22.5", "api_name": "_get_steps", "version": "5.1.6"}


Uploading an estimated of 1 files
Uploading got_persona_dataset_100.csv
Uploaded got_persona_dataset_100.csv, 1 files out of an estimated total of 1
Uploaded 1 files


{
  "definition": "EnginelessDataflow:\n---\ntype: mltable\npaths:\n  - pattern: \"azureml://subscriptions/ebc57341-e881-4473-9659-d8076026bd45/resourcegroups/azure-ai/workspaces/demeyer-robin-ml/datastores/workspaceblobstore/paths/datasets/got_persona_dataset_100.csv\"\ntransformations:\n  - read_delimited:\n      path_column: Path\n      include_path_column: false\n      encoding: utf8\n      support_multi_line: false\n      delimiter: \",\"\n      empty_as_string: false\n      partition_size: 20971520\n      header: all_files_same_headers\n      infer_column_types: false\n  - convert_column_types:\n      - columns: house_affiliation\n        column_type: string\n      - columns: ruthlessness_1to5\n        column_type: int\n      - columns: trait_impulsive\n        column_type: boolean\n      - columns: trait_vengeful\n        column_type: boolean\n      - columns: trait_loyal\n        column_type: boolean\n      - columns: combat_skill_1to5\n        column_type: int\n      - columns

In [14]:
from azure.ai.ml import MLClient, Input, Output, command
from azure.ai.ml.entities import Environment
from azure.identity import DefaultAzureCredential
from azure.ai.ml.dsl import pipeline
from azureml.core import Workspace, Dataset

credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential=credential)
ws = Workspace.from_config()

print(f"✓ Connected to: {ml_client.workspace_name}")

Found the config file in: /config.json


✓ Connected to: demeyer-robin-ml


In [15]:
# Load your existing tabular dataset
dataset = Dataset.get_by_name(ws, name='got-persona-dataset', version='1')
df = dataset.to_pandas_dataframe()

# Save as data.csv (required by your prepare_component.py)
df.to_csv('data.csv', index=False)
print(f"✓ Dataset shape: {df.shape}")

# Upload to datastore
datastore = ws.get_default_datastore()
datastore.upload_files(
    files=['data.csv'],
    target_path='got_input/',
    overwrite=True
)
print("✓ Uploaded to: got_input/data.csv")

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
✓ Dataset shape: (100, 21)
Uploading an estimated of 1 files
Uploading data.csv
Uploaded data.csv, 1 files out of an estimated total of 1
Uploaded 1 files
✓ Uploaded to: got_input/data.csv


In [16]:
env = Environment(
    name="got-prepare-env",
    conda_file="conda.yaml",  # Uses your existing file
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)
env = ml_client.environments.create_or_update(env)
print(f"✓ Environment: {env.name}")

✓ Environment: got-prepare-env


In [17]:
prepare_component = command(
    name="prepare_got_data",
    display_name="Prepare GOT Data",
    inputs={
        "input_folder": Input(type="uri_folder"),
        "target_col": Input(type="string", default="house_affiliation"),
        "test_size": Input(type="number", default=0.2),
        "seed": Input(type="integer", default=42),
        "stratify": Input(type="integer", default=1),
    },
    outputs={
        "out_train": Output(type="uri_folder"),
        "out_test": Output(type="uri_folder"),
    },
    code=".",
    command="""python prepare_component.py \
        --input_folder ${{inputs.input_folder}} \
        --target_col ${{inputs.target_col}} \
        --test_size ${{inputs.test_size}} \
        --seed ${{inputs.seed}} \
        --stratify ${{inputs.stratify}} \
        --out_train ${{outputs.out_train}} \
        --out_test ${{outputs.out_test}}""",
    environment=env
)
print("✓ Component defined")

✓ Component defined


In [18]:
@pipeline(name="got_data_preparation")
def data_prep_pipeline(input_data, target_col="house_affiliation"):
    step = prepare_component(
        input_folder=input_data,
        target_col=target_col,
        test_size=0.2,
        seed=42,
        stratify=1
    )
    return {
        "train_data": step.outputs.out_train,
        "test_data": step.outputs.out_test,
    }

print("✓ Pipeline defined")

✓ Pipeline defined


In [22]:
pipeline_job = data_prep_pipeline(
    input_data=Input(
        type="uri_folder",
        path="azureml://datastores/workspaceblobstore/paths/got_input/"
    ),
    target_col="house_affiliation"
)

# Set default compute for the pipeline
pipeline_job.settings.default_compute = "RDM-compute"

pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job,
    experiment_name="got_data_preparation"
)

print(f"\n✓ Pipeline submitted!")
print(f"View at: {pipeline_job.studio_url}")

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored



✓ Pipeline submitted!
View at: https://ml.azure.com/runs/blue_rose_cb2pw7jml1?wsid=/subscriptions/ebc57341-e881-4473-9659-d8076026bd45/resourcegroups/azure-ai/workspaces/demeyer-robin-ml&tid=4ded4bb1-6bff-42b3-aed7-6a36a503bf7a


In [23]:
# List all components
components = ml_client.components.list()
print("All components:")
for comp in components:
    print(f"  - {comp.name} (version {comp.version})")

All components:
  - training_cli (version None)
  - training (version None)
  - data_split (version None)
  - data_prep_image_resize (version None)


In [24]:
# Register the component
registered_component = ml_client.components.create_or_update(prepare_component)

print(f"✓ Component registered!")
print(f"  Name: {registered_component.name}")
print(f"  Version: {registered_component.version}")
print(f"  ID: {registered_component.id}")

ActivityCompleted: Activity=Component.CreateOrUpdate, HowEnded=Failure, Duration=0.12 [ms], Exception=AttributeError, ErrorCategory=Unknown, ErrorMessage=Got error AttributeError: ''Command' object has no attribute '_is_anonymous'' while calling Component.CreateOrUpdate


AttributeError: 'Command' object has no attribute '_is_anonymous'